In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

# Tutorial: Using datasets

In [ ]:
import os, glob
from rich import print
from image_folder_datasets.core import ImageFolderDataModule

from sklearn import svm
import numpy as np
import pytorch_lightning as pl
from image_folder_datasets.core import CNNModule
from torchvision import transforms as tfms
from einops.layers.torch import Rearrange
from sklearn.metrics import classification_report

data_dirs = sorted(list(glob.glob('datasets/*')))
print(len(data_dirs))

In [ ]:
transform = tfms.Compose([
    tfms.Grayscale(),
    tfms.Resize(128, interpolation=2),
    tfms.RandomCrop(112),
    tfms.ToTensor(),
    Rearrange('h w c -> (h w c)'), 
])

In [ ]:


results = {}

data_dirs = sorted(list(glob.glob('datasets/*')))

for i, data_dir in enumerate(data_dirs):
    
    dm = ImageFolderDataModule(data_dir, 256, transform, num_workers=8)
    dm.setup()
    
    x_train, y_train = zip(*[(np.asarray(x),y) for x, y in dm.trainset])
    x_val, y_val = zip(*[(np.asarray(x),y) for x, y in dm.valset])
    
    clf = svm.SVC()
    clf.fit(x_train, y_train)

    y_pred = clf.predict(x_val)
    result = classification_report(y_val, y_pred, output_dict=True)
    
    dataset_name = os.path.basename(data_dir)
    results[dataset_name] = result
    print(dataset_name, result['accuracy'])
    
   

In [ ]:
results = {}
for i, data_dir in enumerate(data_dirs):
    
    dm = ImageFolderDataModule(data_dir, 256, transform, num_workers=8)
    dm.setup()
    
    x_train, y_train = zip(*[(np.asarray(x),y) for x, y in dm.trainset])
    x_val, y_val = zip(*[(np.asarray(x),y) for x, y in dm.valset])
    
    dummy_clf = DummyClassifier(strategy="most_frequent")
    dummy_clf.fit(x_train, y_train)

    y_pred = dummy_clf.predict(x_val)
    result = classification_report(y_val, y_pred, output_dict=True)
    
    dataset_name = os.path.basename(data_dir)
    results[dataset_name] = result
    print(dataset_name, result['accuracy'])
    break

In [ ]:
from contexttimer import Timer
from sklearn import metrics

def multiclass_report(clf, x_train, y_train, x_val, y_val):
    with Timer() as fit_time:
        clf.fit(x_train, y_train)
        
    with Timer() as inf_time:
        y_pred = clf.predict(x_val)
        
    
    

,0,1,2,3,4,5,6,7,8,9,10,11,accuracy,macro avg,weighted avg
precision,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112981,0.0,0.112981,0.009415,0.012765
recall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.112981,0.083333,0.112981
f1-score,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.203024,0.0,0.112981,0.016919,0.022938
support,67.0,130.0,136.0,141.0,137.0,64.0,68.0,70.0,138.0,70.0,141.000000,86.0,0.112981,1248.000000,1248.000000


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_val, y_pred)

(array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.11298077, 0.        ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.20302376, 0.        ]),
 array([ 67, 130, 136, 141, 137,  64,  68,  70, 138,  70, 141,  86]))